In [ ]:
import pandas as pd

In [ ]:
manual_df = pd.read_excel("/content/LIWC2015_Manual_Transcription_deidentified.xlsx")

In [ ]:
aws_gt_df = pd.read_excel("/content/LIWC2015_AWS_GT_Transcription.xlsx", header=[0, 1])
categories = aws_gt_df.columns.get_level_values(0)
specific_names = aws_gt_df.columns.get_level_values(1)

# Create a mapping of specific names to categories
mapping = {specific_name: category for specific_name, category in zip(specific_names, categories)}

aws_gt_df.columns = aws_gt_df.columns.droplevel(level=0)

In [ ]:
aws_medical_df = pd.read_excel("/content/LIWC_AWS_Medical_Black_White.2.19.24.xlsx")
aws_medical_df.race = aws_medical_df.race.str.title()

In [ ]:
whisper_df = pd.read_excel("/content/LIWC_Whisper_Black_White.19.2.24.xlsx")

In [ ]:
def process_df(df):
    column_names = df.drop(columns=["Fake_id", "race", "WC", "WPS"]).columns
    df[column_names] = df[column_names].multiply(df["WC"] / 100, axis="index")
    df.drop(columns=["Fake_id", "WPS"])
    return df.groupby(by="race").sum()


In [ ]:
manual_df = process_df(manual_df)
aws_gt_df = process_df(aws_gt_df)
aws_medical_df = process_df(aws_medical_df)
whisper_df = process_df(whisper_df)

In [ ]:
manual_general_df = manual_df.copy()
aws_gt_general_df = aws_gt_df.copy()
aws_medical_general_df = aws_medical_df.copy()
whisper_general_df = whisper_df.copy()

In [ ]:
manual_general_df.columns = pd.MultiIndex.from_tuples([(mapping[col], col) for col in manual_general_df.columns])
aws_gt_general_df.columns = pd.MultiIndex.from_tuples([(mapping[col], col) for col in aws_gt_general_df.columns])
aws_medical_general_df.columns = pd.MultiIndex.from_tuples([(mapping[col], col) for col in aws_medical_general_df.columns])
whisper_general_df.columns = pd.MultiIndex.from_tuples([(mapping[col], col) for col in whisper_general_df.columns])

In [ ]:
manual_general_df

Linguistic Dimention  Cognition Conversational                Affect  \
                        WC   Analytic          Clout  Authentic       Tone   
race                                                                         
Black                 5601  1781.4084      1999.9412  3813.1388  1847.1950   
White                 4520  1409.9117      1253.6687  3194.6600  1866.8148   

      Linguistic Dimention                                             ...  \
                       WPS    Sixltr        Dic   function    pronoun  ...   
race                                                                   ...   
Black                 5601  531.9876  5036.0111  3178.0061  1289.9969  ...   
White                 4520  440.0054  4233.9884  2705.0133  1053.9830  ...   

      All punctuation                                                         \
                Comma Colon SemiC QMark Exclam Dash    Quote Apostro Parenth   
race                                                                           
Black             0.0   0.0   0.0   0.0    0.0  0.0  31.0006     0.0     0.0   
White             0.0   0.0   0.0   0.0    0.0  0.0   0.0000     0.0     0.0   

              
      OtherP  
race          
Black    0.0  
White    0.0  

[2 rows x 93 columns]

In [ ]:
manual_general_df = manual_general_df.groupby(axis=1, level=0).sum()
aws_gt_general_df = aws_gt_general_df.groupby(axis=1, level=0).sum()
aws_medical_general_df = aws_medical_general_df.groupby(axis=1, level=0).sum()
whisper_general_df = whisper_general_df.groupby(axis=1, level=0).sum()

In [ ]:
whisper_general_df

,Affect,All punctuation,Cognition,Conversational,Drives,Lifestyle,Linguistic Dimention,Perception,Physical,Social,Time orientation
race,,,,,,,,,,,
Black,1581.5241,0.0,1784.4468,3641.3998,326.9953,61.0083,14773.9866,377.0078,364.9769,410.9965,1182.9896
White,1474.8769,0.0,1580.8584,3168.7733,360.0081,71.0015,13410.9883,331.9870,309.0362,296.9828,1135.9724


In [ ]:
manual_df

,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
race,,,,,,,,,,,,,,,,,,,,,
Black,5601,1781.4084,1999.9412,3813.1388,1847.1950,5601,531.9876,5036.0111,3178.0061,1289.9969,...,0.0,0.0,0.0,0.0,0.0,0.0,31.0006,0.0,0.0,0.0
White,4520,1409.9117,1253.6687,3194.6600,1866.8148,4520,440.0054,4233.9884,2705.0133,1053.9830,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0


In [ ]:
liwc_all = pd.DataFrame({"Manual": manual_df.sum(), "AWS GT": aws_gt_df.sum(), "AWS Medical": aws_medical_df.sum(), "Whisper": whisper_df.sum()})
liwc_general_all = pd.DataFrame({"Manual": manual_general_df.sum(), "AWS GT": aws_gt_general_df.sum(), "AWS Medical": aws_medical_general_df.sum(), "Whisper": whisper_general_df.sum()})


In [ ]:
liwc_all.loc["Quote"] = 0
liwc_all.loc["AllPunc"] = 0

# First Approach

In [ ]:
percentage_of_change_df = pd.DataFrame()
percentage_of_change_df["AWS GT"] = (liwc_all["AWS GT"] - liwc_all["Manual"]) / liwc_all["Manual"]
percentage_of_change_df["AWS Medical"] = (liwc_all["AWS Medical"] - liwc_all["Manual"]) / liwc_all["Manual"]
percentage_of_change_df["Whisper"] = (liwc_all["Whisper"] - liwc_all["Manual"]) / liwc_all["Manual"]


In [ ]:
percentage_of_change_df["mean"] = percentage_of_change_df.mean(axis=1)
percentage_of_change_df = percentage_of_change_df.sort_values(by=("mean"), ascending=False)
percentage_of_change_df.drop(columns=["mean"], inplace=True)
(percentage_of_change_df * 100).round().style.background_gradient(cmap ='coolwarm', axis=None).set_properties(**{'font-size': '12px'}).to_excel('percent_of_change.xlsx', engine='openpyxl')

In [ ]:
percentage_of_change_df = pd.DataFrame()
percentage_of_change_df["AWS GT"] = (liwc_general_all["AWS GT"] - liwc_general_all["Manual"]) / liwc_general_all["Manual"]
percentage_of_change_df["AWS Medical"] = (liwc_general_all["AWS Medical"] - liwc_general_all["Manual"]) / liwc_general_all["Manual"]
percentage_of_change_df["Whisper"] = (liwc_general_all["Whisper"] - liwc_general_all["Manual"]) / liwc_general_all["Manual"]
percentage_of_change_df.drop(["All punctuation"], axis=0, inplace=True)

In [ ]:
percentage_of_change_df["mean"] = percentage_of_change_df.mean(axis=1)
percentage_of_change_df = percentage_of_change_df.sort_values(by=("mean"), ascending=False)
percentage_of_change_df.drop(columns=["mean"], inplace=True)
(percentage_of_change_df * 100).round().style.background_gradient(cmap ='coolwarm', axis=None).set_properties(**{'font-size': '12px'}).to_excel('general_%_change.xlsx', engine='openpyxl')

In [ ]:
liwc_gp = pd.concat([manual_df.T, aws_gt_df.T, aws_medical_df.T], axis=1, keys=["Manual", "AWS GT", "AWS Medical"])

In [ ]:
liwc_general_gp = pd.concat([manual_general_df.T, aws_gt_general_df.T, aws_medical_general_df.T],
                            axis=1, keys=["Manual", "AWS GT", "AWS Medical"])

In [ ]:
liwc_general_gp

Manual                  AWS GT              \
race                       Black       White       Black       White   
Affect                 2178.2021   2375.8020   2924.8434   2730.8045   
All punctuation          62.0012      0.0000      0.0000      0.0000   
Cognition              3269.3344   2748.9008   3342.6332   2996.9402   
Conversational         6496.0571   4989.2871   7208.0736   5712.5494   
Drives                  564.9790    421.9732    667.9541    520.9570   
Lifestyle               104.9854    117.9816    110.9937    123.9964   
Linguistic Dimention  27201.8060  22475.8916  28971.8594  24770.8631   
Perception              692.9497    494.9632    770.9458    545.9811   
Physical                540.9959    427.9769    552.9845    415.9984   
Social                  675.9890    392.9931    799.9820    500.9856   
Time orientation       2216.9495   1920.9688   2385.9523   2051.9650   

                     AWS Medical              
race                       Black       White  
Affect                 4063.4279   3125.6601  
All punctuation           0.0000      0.0000  
Cognition              4299.3217   3294.7952  
Conversational         9405.5355   6757.9075  
Drives                  829.9946    618.0238  
Lifestyle               141.9828    120.9896  
Linguistic Dimention  37314.7890  28234.9994  
Perception              974.9721    711.9630  
Physical                760.9827    440.9730  
Social                 1018.9868    610.0028  
Time orientation       3036.9363   2466.0045

In [ ]:
liwc_gp.loc["prep", :]

             race 
Manual       Black    538.9892
             White    447.9979
AWS GT       Black    597.9923
             White    487.9973
AWS Medical  Black    803.9731
             White    542.9844
Name: prep, dtype: float64

In [ ]:
percentage_of_change_df = pd.DataFrame(columns=liwc_general_gp.columns)
# percentage_of_change_df.columns = liwc_gp.columns
manual_black = ("Manual", "Black")
gt_black = ("AWS GT", "Black")
medical_black = ("AWS Medical", "Black")

manual_white = ("Manual", "White")
gt_white = ("AWS GT", "White")
medical_white = ("AWS Medical", "White")

percentage_of_change_df[gt_black] = (liwc_general_gp[gt_black] - liwc_general_gp[manual_black]) / liwc_general_gp[manual_black]
percentage_of_change_df[medical_black] = (liwc_general_gp[medical_black] - liwc_general_gp[manual_black]) / liwc_general_gp[manual_black]


percentage_of_change_df[gt_white] = (liwc_general_gp[gt_white] - liwc_general_gp[manual_white]) / liwc_general_gp[manual_white]
percentage_of_change_df[medical_white] = (liwc_general_gp[medical_white] - liwc_general_gp[manual_white]) / liwc_general_gp[manual_white]


In [ ]:
percentage_of_change_df.drop(columns=["Manual"], inplace=True)

In [ ]:
percentage_of_change_df

AWS GT           AWS Medical          
race                     Black     White       Black     White
Affect                0.342779  0.149424    0.865496  0.315623
All punctuation      -1.000000       NaN   -1.000000       NaN
Cognition             0.022420  0.090232    0.315045  0.198586
Conversational        0.109607  0.144963    0.447884  0.354484
Drives                0.182264  0.234574    0.469072  0.464604
Lifestyle             0.057230  0.050981    0.352405  0.025496
Linguistic Dimention  0.065071  0.102108    0.371776  0.256235
Perception            0.112557  0.103074    0.406988  0.438416
Physical              0.022160 -0.027989    0.406633  0.030366
Social                0.183425  0.274795    0.507401  0.552197
Time orientation      0.076232  0.068193    0.369872  0.283730

In [ ]:
import numpy as np
percentage_of_change_df.replace([np.inf, -np.inf], np.nan, inplace=True)
percentage_of_change_df["mean"] = percentage_of_change_df.mean(axis=1)
percentage_of_change_df = percentage_of_change_df.sort_values(by=("mean"), ascending=False)
percentage_of_change_df.drop(columns=["mean"], inplace=True)
(percentage_of_change_df * 100).round().style.background_gradient(cmap ='coolwarm', axis=None).set_properties(**{'font-size': '12px'}).to_excel('general_groups.xlsx', engine='openpyxl')